In [17]:
import os
from os import path
import cv2
import glob
import re
import numpy as np
import matplotlib.pyplot as plt

In [18]:
if (path.exists('chroma')!= 1):
        os.mkdir('chroma')

if (path.exists('img')!= 1):
        os.mkdir('img')

if (path.exists('webcam')!= 1):
        os.mkdir('webcam')

if (path.exists('sky')!= 1):
        os.mkdir('sky')


In [3]:
def vid_to_img(vid_path,img_path):
    # Funtion to convert video to frames
    num = 0
    exist = True
    while exist == True:
        capture = cv2.VideoCapture(vid_path)
        capture.set(cv2.CAP_PROP_POS_MSEC,(num*50))
        exist, img = capture.read()
        num = num +1
        cv2.imwrite(img_path+"/img_%d.png" % num, img)
    os.remove(img_path+'/img_' + str(num) + '.png')

vid_to_img("Bird.mov","img")
    
    

In [19]:
def img_to_vid(img_path,vid_path,fps = 30):
    # Function to convert sequence of frames to video
    images = []
    size = (1920,1080)
    fr = glob.glob(img_path)
    fr.sort(key=lambda f: int(re.sub('\D', '', f)))
    for img in fr:
        image = cv2.imread(img,1)
#         cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
        images.append(image)
#         size = (img.shape[0],img.shape[1])
        
    write = cv2.VideoWriter_fourcc(*'XVID')
    vid = cv2.VideoWriter(vid_path, write, fps, size)
    i = 0
    length = len(images)
    while i<length:
        vid.write(images[i])
#         cv2.imshow("frame",images[i])
        i = i+1
    vid.release()
img_to_vid("img/*.png","video.avi")


In [5]:
def webcam():
    # Function to convert webcam input video to frames
    num = 0
    cam = cv2.VideoCapture(0)
    while True:
        num = num + 1
        exist, img = cam.read()
        cv2.imshow("Image",img)
        cv2.imwrite("webcam/img_%d.png" % num, img)
        if(cv2.waitKey(1) & 0xFF == ord('s')):
            break
    cv2.destroyAllWindows()
    cam.release()
webcam()

In [6]:
def chroma_key(img,bg_image):
    # Function to to chroma keying for one frame
    bg_im = cv2.cvtColor(bg_image, cv2.COLOR_BGR2RGB)
    cop = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    green1 = np.array([0, 100, 0])
    green2 = np.array([120, 255, 100])
    mask = cv2.inRange(cop, green1, green2)
    cop[mask!=0] = [0,0,0]
    bg_resize = bg_im[0:1080, 0:1920]
    bg_resize[mask == 0] = [0, 0, 0]
    return bg_resize + cop

In [7]:
img_path = "img/*.png"
bg_image = vid_to_img("blue_sky.mov","sky")

In [14]:
# bg_image = "images.jpeg"
fr = glob.glob(img_path)
fr.sort(key=lambda f: int(re.sub('\D', '', f)))

fr1 = glob.glob("sky/*.png")
fr1.sort(key=lambda f: int(re.sub('\D', '', f)))
num = 0
x = min(len(fr),len(fr1))
for i in range(x):
    num = num + 1
    image = cv2.imread(fr[i],1)
    bg_im = cv2.imread(fr1[i],1)
    chroma = chroma_key(image,bg_im)
    chroma = cv2.cvtColor(chroma, cv2.COLOR_RGB2BGR)
    cv2.imwrite("chroma/chroma_%d.png" % num, chroma)

In [15]:
img_to_vid("chroma/*.png", "chroma_video.avi")